In [1]:
import pandas as pd
import numpy as np
import acquire
print('Success')

Success


In [2]:
telco = acquire.get_telco()
telco.dtypes

customer_id                    object
gender                         object
senior_citizen                  int64
partner                        object
dependents                     object
tenure                          int64
phone_service                  object
multiple_lines                 object
internet_service_type_id        int64
online_security                object
online_backup                  object
device_protection              object
tech_support                   object
streaming_tv                   object
streaming_movies               object
contract_type_id                int64
paperless_billing              object
payment_type_id                 int64
monthly_charges               float64
total_charges                  object
churn                          object
internet_service_type_id.1      int64
internet_service_type          object
payment_type_id.1               int64
payment_type                   object
contract_type_id.1              int64
contract_typ

In [3]:
telco.drop(columns=['contract_type_id.1', 'internet_service_type_id.1', 'payment_type_id.1'], inplace = True)
telco['total_charges'] = pd.to_numeric(telco['total_charges'],errors='coerce')
telco.dropna(inplace=True)
telco['monthly_charges'] = round(telco['total_charges']/telco['tenure'], 2)
telco['is_alone'] = (telco['dependents'] == 0) & (telco['partner'] == 0)
telco['tenure_year'] = round(telco['tenure']/12, 2)
telco['additional_services'] = (telco['tech_support'] == 'Yes') | (telco['online_security'] == 'Yes') | (telco['online_backup'] == 'Yes') | (telco['device_protection'] == 'Yes')
telco['streamer'] = (telco['streaming_tv'] == 'Yes') | (telco['streaming_movies'] == 'Yes')
telco['has_internet'] = telco['internet_service_type'] != 'None'
telco['auto_pay'] = telco['payment_type'].str.contains('auto')
telco.replace(False, 0, inplace=True)
telco.replace(True, 1, inplace=True)
telco.replace('No', 0, inplace=True)
telco.replace('Yes', 1, inplace=True)
telco.replace('No phone service', 0, inplace=True)
telco.replace('No internet service', 0, inplace=True)
telco

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,churn,internet_service_type,payment_type,contract_type,is_alone,tenure_year,additional_services,streamer,has_internet,auto_pay
0,0003-MKNFE,Male,0,0,0,9,1,1,1,0,...,0,DSL,Mailed check,Month-to-month,0.0,0.75,0.0,1.0,1.0,0.0
1,0013-MHZWF,Female,0,0,1,9,1,0,1,0,...,0,DSL,Credit card (automatic),Month-to-month,0.0,0.75,1.0,1.0,1.0,1.0
2,0015-UOCOJ,Female,1,0,0,7,1,0,1,1,...,0,DSL,Electronic check,Month-to-month,0.0,0.58,1.0,0.0,1.0,0.0
3,0023-HGHWL,Male,1,0,0,1,0,0,1,0,...,1,DSL,Electronic check,Month-to-month,0.0,0.08,0.0,0.0,1.0,0.0
4,0032-PGELS,Female,0,1,1,1,0,0,1,1,...,1,DSL,Bank transfer (automatic),Month-to-month,0.0,0.08,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,9950-MTGYX,Male,0,1,1,28,1,0,3,0,...,0,None,Credit card (automatic),Two year,0.0,2.33,0.0,0.0,0.0,1.0
7039,9953-ZMKSM,Male,0,0,0,63,1,1,3,0,...,0,None,Mailed check,Two year,0.0,5.25,0.0,0.0,0.0,0.0
7040,9964-WBQDJ,Female,0,1,0,71,1,1,3,0,...,0,None,Credit card (automatic),Two year,0.0,5.92,0.0,0.0,0.0,1.0
7041,9972-EWRJS,Female,0,1,1,67,1,0,3,0,...,0,None,Bank transfer (automatic),Two year,0.0,5.58,0.0,0.0,0.0,1.0


In [4]:
from sklearn.model_selection import train_test_split
train_validate, test = train_test_split(telco, test_size=.2, random_state=333, stratify=telco.churn)
train, validate = train_test_split(train_validate, test_size=.25, random_state=333, stratify=train_validate.churn)
print(train.shape[0], validate.shape[0], test.shape[0])

4218 1407 1407


In [7]:
base = train
base['baseline'] = 0
base_accuracy = (base['churn'] == base['baseline']).mean()
base_accuracy

<ipython-input-7-7afd6cf8c3d4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base['baseline'] = 0


0.7342342342342343

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


X_train = train[['streamer', 'auto_pay', 'additional_services']]
y_train = train[['churn']]

clf = DecisionTreeClassifier(max_depth=3, random_state=333)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_train)
y_pred_proba = clf.predict_proba(X_train)
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print(confusion_matrix(y_train, y_pred))
print(classification_report(y_train, y_pred))

Accuracy of Decision Tree classifier on training set: 0.75
[[2973  124]
 [ 914  207]]
              precision    recall  f1-score   support

           0       0.76      0.96      0.85      3097
           1       0.63      0.18      0.29      1121

    accuracy                           0.75      4218
   macro avg       0.70      0.57      0.57      4218
weighted avg       0.73      0.75      0.70      4218

